In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
# Load dataset
mnist = input_data.read_data_sets('./dataset/MNIST_data',one_hot=True)

# param set
batch_size = 100                                  #单批次数据量
n_batch = mnist.train.num_examples // batch_size  #批次数量
hidden_layer1_node = 200                         #隐藏层1节点数
hidden_layer2_node = 50                         #隐藏层2节点数
output_layer_node = 10                            #输出层节点数
steps = 30                                        #训练迭代次数
lr = 0.1                                          #学习速率
regularization_rate = 0.0012                        #正则化系数

# 训练数据插入点
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32,[None,784])
    y = tf.placeholder(tf.float32,[None,output_layer_node])
    keep_prob = tf.placeholder(tf.float32)

def inference(input_tensor,keep_prob,regularization_rate,reuse=False):
# weight & biases
    with tf.variable_scope('hidden_layer1',reuse=reuse):
        hidden_layer1_w = tf.get_variable('w',[784,hidden_layer1_node],
                                          initializer=tf.truncated_normal_initializer(stddev=0.1))
        hidden_layer1_b = tf.get_variable('b',hidden_layer1_node,
                                          initializer=tf.constant_initializer(0.1))
        hidden_layer1 = tf.nn.relu(tf.matmul(x,hidden_layer1_w) + hidden_layer1_b)
        hidden_layer1 = tf.nn.dropout(x=hidden_layer1,keep_prob=keep_prob)
        
    with tf.variable_scope('hidden_layer2',reuse=reuse):
        hidden_layer2_w = tf.get_variable('w',[hidden_layer1_node,hidden_layer2_node],
                                          initializer=tf.truncated_normal_initializer(stddev=0.1))
        hidden_layer2_b = tf.get_variable('b',hidden_layer2_node,
                                          initializer=tf.constant_initializer(0.1))
        hidden_layer2 = tf.nn.relu(tf.matmul(hidden_layer1,hidden_layer2_w) + hidden_layer2_b)
        hidden_layer2 = tf.nn.dropout(x=hidden_layer2,keep_prob=keep_prob)
        
    with tf.variable_scope('out_layer',reuse=reuse):
        out_layer_w = tf.get_variable('w',[hidden_layer2_node,output_layer_node],
                                      initializer=tf.truncated_normal_initializer(stddev=0.1))
        out_layer_b = tf.get_variable('b',output_layer_node,
                                      initializer=tf.constant_initializer(0.1))
        prediction = tf.matmul(hidden_layer2,out_layer_w) + out_layer_b
    with tf.variable_scope('regular',reuse=reuse):
        regularizer = tf.contrib.layers.l2_regularizer(regularization_rate)
        regularization = regularizer(hidden_layer1_w) + regularizer(hidden_layer2_w) + regularizer(out_layer_w)
    return prediction,regularization

# # 创建神经网络
#     with tf.name_scope('hidden_layer1'):
#         hidden_layer1 = tf.nn.relu(tf.matmul(x,hidden_layer1_w) + hidden_layer1_b)
#         hidden_layer1 = tf.nn.dropout(x=hidden_layer1,keep_prob=keep_prob)
#     with tf.name_scope('hidden_layer2'):
#         hidden_layer2 = tf.nn.relu(tf.matmul(hidden_layer1,hidden_layer2_w) + hidden_layer2_b)
#         #hidden_layer2 = tf.nn.dropout(x=hidden_layer2,keep_prob=keep_prob)
#     with tf.name_scope('out_layer'):    
#         prediction = tf.matmul(hidden_layer2,out_layer_w) + out_layer_b

#L2正则化
# with tf.name_scope('regular'):
#     regularizer = tf.contrib.layers.l2_regularizer(regularization_rate)
#     regularization = regularizer(hidden_layer1_w) + regularizer(hidden_layer2_w) + regularizer(out_layer_w)

prediction,regularization = inference(x,keep_prob,regularization_rate) 

# 损失函数
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=tf.argmax(y,1),logits=prediction)) + regularization

# 定义训练方式
with tf.name_scope('train_step'):
    train_step = tf.train.GradientDescentOptimizer(lr).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

#准确率
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

writer = tf.summary.FileWriter('./tensorborad/logs',tf.get_default_graph())
writer.close()

# runtime
with tf.Session() as sess:
    #初始化变量
    sess.run(init)
    for i in range(steps):
        for batch in range(n_batch):
            x_train,y_train = mnist.train.next_batch(batch)
            sess.run(train_step,feed_dict={x:x_train,y:y_train,keep_prob:0.8})
        if (i+1) % 2 ==0:
            acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
            train_acc = sess.run(accuracy,feed_dict={x:mnist.train.images,y:mnist.train.labels,keep_prob:1.0})
            print('Iter %d, Testing Accuracy: %f,Train Accuracy:%f' % (i+1,acc,train_acc))


Extracting ./dataset/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./dataset/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./dataset/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./dataset/MNIST_data/t10k-labels-idx1-ubyte.gz
Iter 2, Testing Accuracy: 0.944300,Train Accuracy:0.945091
Iter 4, Testing Accuracy: 0.959900,Train Accuracy:0.962527
Iter 6, Testing Accuracy: 0.966800,Train Accuracy:0.971236
Iter 8, Testing Accuracy: 0.971300,Train Accuracy:0.976745
Iter 10, Testing Accuracy: 0.971700,Train Accuracy:0.977836
Iter 12, Testing Accuracy: 0.973900,Train Accuracy:0.982018
Iter 14, Testing Accuracy: 0.976200,Train Accuracy:0.984145
Iter 16, Testing Accuracy: 0.976300,Train Accuracy:0.985545
Iter 18, Testing Accuracy: 0.978400,Train Accuracy:0.986182
Iter 20, Testing Accuracy: 0.979000,Train Accuracy:0.987564
Iter 22, Testing Accuracy: 0.979400,Train Accuracy:0.988582
Iter 24, Testing Accuracy: 0.979700,Train Accuracy:0.988655
Iter 26, Testing Accuracy: 0.980200,Train Accuracy